In [ ]:
def cal_frequency(text, api_data_class, stop_word='default'):
    #determine language
    if stop_word == 'default':
        lang = api_data_class.get_lang(text)
        if lang == 'zho':
            stop_word = 'pacakge_data/cn_stopwords.txt'
        elif lang == 'eng':
            stop_word = 'pacakge_data/en_stopwords.txt'
        else:
            stop_word = 'pacakge_data/other_stopwords.txt'

    try:
        stop_file = open(stop_word, 'r', encoding='utf-8')
        stopwords = stop_file.read().split("\n")
        stop_file.close()
    except:
        print('{} is not exist, please check the file!'.format(stop_word))

    #check the characters limit (need a function)
    text_lst = check_limit(text)
    text_token = []
    for t in text_lst:
        text_token.expend(api_data_class.get_token(t))

    washed_token = []
    for char in text_token:
        if char in stopwords:
            pass
        else:
            wasked_token.append(char)

    counts = {}
    for word in washed_token:
        counts[word] = counts.get(word, 0) + 1

    counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    return counts

In [36]:
def check_limit(text):
    text = text.replace('\n', ' ')
    if len(text.replace(' ', '')) <= 5000:
        return [text]
    else:
        true_text = []
        if '.' in text:
            true_text = check_limit_tool(text, '.')
        elif '。' in text:
            true_text = check_limit_tool(text, '。')
        else:
            true_text = check_limit_tool(text, '')
    return true_text


def check_limit_tool(text, symbol):
    temp = []
    string = ''
    if symbol != '':
        split_text = text.split(symbol)
    else:
        split_text = text
    for sen in split_text:
        temp_s = string + sen + symbol
        if len(temp_s.replace(' ', '')) > 5000:
            temp.append(string)
            string = sen + symbol
        else:
            string = string + sen + symbol

        if sen == split_text[-1]:
            temp.append(string)
    return temp

In [ ]:
#file_loc: where the wordcloud pic and chart pic should be saved
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
from altair_saver import save


def word_viz(text,
             file_loc,
             api_data_class,
             top_num,
             stop_word='default',
             font_path='pacakge_data/simsunb.ttf',
             cloud_set=WordCloud(font_path=font_path)):
    #determine language
    if stop_word == 'default':
        lang = api_data_class.get_lang(text)
        if lang == 'zho':
            stop_word = 'pacakge_data/cn_stopwords.txt'
        elif lang == 'eng':
            stop_word = 'pacakge_data/en_stopwords.txt'
        else:
            stop_word = 'pacakge_data/other_stopwords.txt'

    try:
        stop_file = open(stop_word, 'r', encoding='utf-8')
        stopwords = stop_file.read().split("\n")
        stop_file.close()
    except:
        print('{} is not exist, please check the file!'.format(stop_word))

    #check the characters limit (need a function)
    text_lst = check_limit(text)
    text_token = []
    for t in text_lst:
        text_token.expend(api_data_class.get_token(t))

    washed_token = []
    for char in text_token:
        if char in stopwords:
            pass
        else:
            wasked_token.append(char)

    cloud_set.generate(' '.join(washed_token))
    cloud_set.to_file(file_loc + 'cloud.png')

    counts = {}
    for word in washed_token:
        counts[word] = counts.get(word, 0) + 1

    counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    words = []
    count = []
    for i in counts:
        words.append(i[0])
        count.append(i[1])
    df = pd.DataFrame({'words': words, 'count': count})
    chart = (alt.Chart(df[:top_num]).mark_line().encode(
        x='words', y='words').properties(height=400, width=400))
    save(chart, "chart.html")